# Classification Exercise

We'll be working with some California Census Data, we'll be trying to use various features of an individual to predict what class of income they belogn in (>50k or <=50k). 

Here is some information about the data:

<table>
<thead>
<tr>
<th>Column Name</th>
<th>Type</th>
<th>Description</th>
</tr>
</thead>
<tbody>
<tr>
<td>age</td>
<td>Continuous</td>
<td>The age of the individual</td>
</tr>
<tr>
<td>workclass</td>
<td>Categorical</td>
<td>The type of employer the  individual has (government,  military, private, etc.).</td>
</tr>
<tr>
<td>fnlwgt</td>
<td>Continuous</td>
<td>The number of people the census  takers believe that observation  represents (sample weight). This  variable will not be used.</td>
</tr>
<tr>
<td>education</td>
<td>Categorical</td>
<td>The highest level of education  achieved for that individual.</td>
</tr>
<tr>
<td>education_num</td>
<td>Continuous</td>
<td>The highest level of education in  numerical form.</td>
</tr>
<tr>
<td>marital_status</td>
<td>Categorical</td>
<td>Marital status of the individual.</td>
</tr>
<tr>
<td>occupation</td>
<td>Categorical</td>
<td>The occupation of the individual.</td>
</tr>
<tr>
<td>relationship</td>
<td>Categorical</td>
<td>Wife, Own-child, Husband,  Not-in-family, Other-relative,  Unmarried.</td>
</tr>
<tr>
<td>race</td>
<td>Categorical</td>
<td>White, Asian-Pac-Islander,  Amer-Indian-Eskimo, Other, Black.</td>
</tr>
<tr>
<td>gender</td>
<td>Categorical</td>
<td>Female, Male.</td>
</tr>
<tr>
<td>capital_gain</td>
<td>Continuous</td>
<td>Capital gains recorded.</td>
</tr>
<tr>
<td>capital_loss</td>
<td>Continuous</td>
<td>Capital Losses recorded.</td>
</tr>
<tr>
<td>hours_per_week</td>
<td>Continuous</td>
<td>Hours worked per week.</td>
</tr>
<tr>
<td>native_country</td>
<td>Categorical</td>
<td>Country of origin of the  individual.</td>
</tr>
<tr>
<td>income</td>
<td>Categorical</td>
<td>"&gt;50K" or "&lt;=50K", meaning  whether the person makes more  than \$50,000 annually.</td>
</tr>
</tbody>
</table>

## Follow the Directions in Bold. If you get stuck, check out the solutions lecture.

### THE DATA

** Read in the census_data.csv data with pandas**

In [1]:
import pandas as pd
from sklearn import preprocessing, model_selection, metrics

In [2]:
df = pd.read_csv('census_data.csv')

In [3]:
df.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


** TensorFlow won't be able to understand strings as labels, you'll need to use pandas .apply() method to apply a custom function that converts them to 0s and 1s. This might be hard if you aren't very familiar with pandas, so feel free to take a peek at the solutions for this part.**

** Convert the Label column to 0s and 1s instead of strings.**

In [4]:
df['income_bracket'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [5]:
lb = preprocessing.LabelBinarizer()
df['income_bracket'] = lb.fit_transform(df['income_bracket'])

In [6]:
df = df.dropna(axis=0)

In [7]:
cols_to_norm = ['age', 'education_num', 'capital_gain', 'capital_loss', 'hours_per_week']
df[cols_to_norm] = df[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

### Perform a Train Test Split on the Data

In [8]:
x_data = df.drop('income_bracket',axis=1)
labels = df['income_bracket']

In [9]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(x_data, labels, test_size=0.3, random_state=101)

### Create the Feature Columns for tf.esitmator

** Take note of categorical vs continuous values! **

In [10]:
df.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

** Import Tensorflow **

In [11]:
import tensorflow as tf

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


** Create the tf.feature_columns for the categorical values. Use vocabulary lists or just use hash buckets. **

In [12]:
workclass = tf.feature_column.categorical_column_with_hash_bucket('workclass', hash_bucket_size=df['workclass'].unique().size)
education = tf.feature_column.categorical_column_with_hash_bucket('education', hash_bucket_size=df['education'].unique().size)
marital = tf.feature_column.categorical_column_with_hash_bucket('marital_status', hash_bucket_size=df['marital_status'].unique().size)
occupation = tf.feature_column.categorical_column_with_hash_bucket('occupation', hash_bucket_size=df['occupation'].unique().size)
relation = tf.feature_column.categorical_column_with_hash_bucket('relationship', hash_bucket_size=df['relationship'].unique().size)
race = tf.feature_column.categorical_column_with_hash_bucket('race', hash_bucket_size=df['race'].unique().size)
country = tf.feature_column.categorical_column_with_hash_bucket('native_country', hash_bucket_size=df['native_country'].unique().size)
gender = tf.feature_column.categorical_column_with_hash_bucket('gender', hash_bucket_size=df['gender'].unique().size)

** Create the continuous feature_columns for the continuous values using numeric_column **

In [13]:
age = tf.feature_column.numeric_column('age')
edu_num = tf.feature_column.numeric_column('education_num')
gain = tf.feature_column.numeric_column('capital_gain')
loss = tf.feature_column.numeric_column('capital_loss')
hours = tf.feature_column.numeric_column('hours_per_week')

** Put all these variables into a single list with the variable name feat_cols **

In [14]:
feat_cols = [age, workclass, education, edu_num, marital, occupation, relation, race, gender, gain, loss, hours, country]

### Create Input Function

** Batch_size is up to you. But do make sure to shuffle!**

In [15]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train, batch_size=10, num_epochs=1000, shuffle=True)

#### Create your model with tf.estimator

**Create a LinearClassifier.(If you want to use a DNNClassifier, keep in mind you'll need to create embedded columns out of the cateogrical feature that use strings, check out the previous lecture on this for more info.)**

In [16]:
model = tf.estimator.LinearClassifier(feature_columns=feat_cols,n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Julia\\AppData\\Local\\Temp\\tmp2o_x6f9q', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000024C36F67CF8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


** Train your model on the data, for at least 5000 steps. **

In [17]:
model.train(input_fn=input_func,steps=5000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\Julia\AppData\Local\Temp\tmp2o_x6f9q\model.ckpt.
INFO:tensorflow:loss = 6.931472, step = 1
INFO:tensorflow:global_step/sec: 84.2177
INFO:tensorflow:loss = 3.545757, step = 101 (1.194 sec)
INFO:tensorflow:global_step/sec: 208.075
INFO:tensorflow:loss = 1.789376, step = 201 (0.479 sec)
INFO:tensorflow:global_step/sec: 209.788
INFO:tensorflow:loss = 3.399306, step = 301 (0.477 sec)
INFO:tensorflow:global_step/sec: 206.436
INFO:tensorflow:loss = 4.753238, step = 401 (0.484 sec)
INFO:tensorflow:global_step/sec: 211.89
INFO:tensorflow:loss = 2.4346957, step = 501 (0.472 sec)
INFO:tensorflow:global_step/sec: 203
INFO:tensorflow:loss = 4.933132, step = 601 (0.492 sec)
INFO:tensorflow:global_step/sec: 212.61

### Evaluation

** Create a prediction input function. Remember to only supprt X_test data and keep shuffle=False. **

In [18]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test, y=y_test, batch_size=10, num_epochs=1, shuffle=False)

** Use model.predict() and pass in your input function. This will produce a generator of predictions, which you can then transform into a list, with list() **

In [19]:
results = model.predict(eval_input_func)
res = list(results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Julia\AppData\Local\Temp\tmp2o_x6f9q\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


** Each item in your list will look like this: **

In [20]:
for result in res:
    print(result)

{'logits': array([-1.1955004], dtype=float32), 'logistic': array([0.23227665], dtype=float32), 'probabilities': array([0.7677234 , 0.23227665], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-3.552964], dtype=float32), 'logistic': array([0.02784223], dtype=float32), 'probabilities': array([0.9721577 , 0.02784223], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-1.087449], dtype=float32), 'logistic': array([0.25209895], dtype=float32), 'probabilities': array([0.7479011 , 0.25209898], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-3.069223], dtype=float32), 'logistic': array([0.04439478], dtype=float32), 'probabilities': array([0.95560515, 0.04439478], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-1.6963272], dtype=float32), 'log

{'logits': array([-1.8342273], dtype=float32), 'logistic': array([0.13773546], dtype=float32), 'probabilities': array([0.8622646 , 0.13773544], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-4.320437], dtype=float32), 'logistic': array([0.01311966], dtype=float32), 'probabilities': array([0.9868803 , 0.01311966], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-3.0125213], dtype=float32), 'logistic': array([0.0468634], dtype=float32), 'probabilities': array([0.9531366, 0.0468634], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-1.7208607], dtype=float32), 'logistic': array([0.15176032], dtype=float32), 'probabilities': array([0.8482397 , 0.15176034], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-1.486521], dtype=float32), 'logis

{'logits': array([-0.72868866], dtype=float32), 'logistic': array([0.32548255], dtype=float32), 'probabilities': array([0.67451745, 0.32548258], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-0.978334], dtype=float32), 'logistic': array([0.27322248], dtype=float32), 'probabilities': array([0.72677755, 0.27322248], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-0.4433636], dtype=float32), 'logistic': array([0.3909398], dtype=float32), 'probabilities': array([0.60906017, 0.39093974], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-1.4373255], dtype=float32), 'logistic': array([0.19195986], dtype=float32), 'probabilities': array([0.80804014, 0.19195984], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-2.932563], dtype=float32), 'lo

{'logits': array([-3.1532], dtype=float32), 'logistic': array([0.04096538], dtype=float32), 'probabilities': array([0.9590346 , 0.04096538], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-2.130856], dtype=float32), 'logistic': array([0.10613375], dtype=float32), 'probabilities': array([0.8938663 , 0.10613376], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-4.4621696], dtype=float32), 'logistic': array([0.01140571], dtype=float32), 'probabilities': array([0.9885943 , 0.01140571], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-5.3000326], dtype=float32), 'logistic': array([0.00496664], dtype=float32), 'probabilities': array([0.9950334 , 0.00496664], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-2.0470827], dtype=float32), 'logi

{'logits': array([-2.8048553], dtype=float32), 'logistic': array([0.05706237], dtype=float32), 'probabilities': array([0.9429376 , 0.05706236], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-1.4107866], dtype=float32), 'logistic': array([0.19611001], dtype=float32), 'probabilities': array([0.80389   , 0.19611001], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-2.4827714], dtype=float32), 'logistic': array([0.07707483], dtype=float32), 'probabilities': array([0.9229252 , 0.07707483], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-1.5254829], dtype=float32), 'logistic': array([0.17865555], dtype=float32), 'probabilities': array([0.8213445 , 0.17865556], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-2.5718849], dtype=float32), '

...

{'logits': array([-1.7729564], dtype=float32), 'logistic': array([0.14517507], dtype=float32), 'probabilities': array([0.8548249 , 0.14517505], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-1.2507336], dtype=float32), 'logistic': array([0.22257316], dtype=float32), 'probabilities': array([0.77742684, 0.22257318], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-3.9474525], dtype=float32), 'logistic': array([0.01893823], dtype=float32), 'probabilities': array([0.98106176, 0.01893824], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([2.275672], dtype=float32), 'logistic': array([0.90684205], dtype=float32), 'probabilities': array([0.09315795, 0.90684205], dtype=float32), 'class_ids': array([1], dtype=int64), 'classes': array([b'1'], dtype=object)}
{'logits': array([-1.979218], dtype=float32), 'log

{'logits': array([-1.495679], dtype=float32), 'logistic': array([0.18307087], dtype=float32), 'probabilities': array([0.8169291 , 0.18307085], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([1.2383006], dtype=float32), 'logistic': array([0.775268], dtype=float32), 'probabilities': array([0.22473192, 0.775268  ], dtype=float32), 'class_ids': array([1], dtype=int64), 'classes': array([b'1'], dtype=object)}
{'logits': array([-2.2667882], dtype=float32), 'logistic': array([0.09391115], dtype=float32), 'probabilities': array([0.9060889 , 0.09391116], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-1.8671954], dtype=float32), 'logistic': array([0.13386658], dtype=float32), 'probabilities': array([0.86613345, 0.13386658], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-1.3933916], dtype=float32), 'logi

{'logits': array([-1.5316148], dtype=float32), 'logistic': array([0.17775756], dtype=float32), 'probabilities': array([0.82224244, 0.17775753], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-1.0696208], dtype=float32), 'logistic': array([0.2554752], dtype=float32), 'probabilities': array([0.74452484, 0.25547522], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-1.1761906], dtype=float32), 'logistic': array([0.23573783], dtype=float32), 'probabilities': array([0.76426214, 0.2357378 ], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-0.39567572], dtype=float32), 'logistic': array([0.40235174], dtype=float32), 'probabilities': array([0.59764826, 0.40235174], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-2.7645404], dtype=float32), '

{'logits': array([0.01982242], dtype=float32), 'logistic': array([0.5049555], dtype=float32), 'probabilities': array([0.49504456, 0.5049555 ], dtype=float32), 'class_ids': array([1], dtype=int64), 'classes': array([b'1'], dtype=object)}
{'logits': array([-1.6153042], dtype=float32), 'logistic': array([0.16585349], dtype=float32), 'probabilities': array([0.83414656, 0.1658535 ], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-2.8283796], dtype=float32), 'logistic': array([0.05580972], dtype=float32), 'probabilities': array([0.9441903 , 0.05580973], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([1.8638582], dtype=float32), 'logistic': array([0.865746], dtype=float32), 'probabilities': array([0.13425398, 0.865746  ], dtype=float32), 'class_ids': array([1], dtype=int64), 'classes': array([b'1'], dtype=object)}
{'logits': array([-3.7218795], dtype=float32), 'logi

{'logits': array([-3.3739097], dtype=float32), 'logistic': array([0.03312087], dtype=float32), 'probabilities': array([0.96687907, 0.03312087], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-2.5670943], dtype=float32), 'logistic': array([0.07128644], dtype=float32), 'probabilities': array([0.92871356, 0.07128643], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-1.1303594], dtype=float32), 'logistic': array([0.24409477], dtype=float32), 'probabilities': array([0.7559052 , 0.24409476], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-2.6884422], dtype=float32), 'logistic': array([0.06365881], dtype=float32), 'probabilities': array([0.9363412 , 0.06365881], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-0.7172095], dtype=float32), '

** Create a list of only the class_ids key values from the prediction list of dictionaries, these are the predictions you will use to compare against the real y_test values. **

In [21]:
class_ids = [result['class_ids'][0] for result in res]

In [22]:
class_ids

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,


** Import classification_report from sklearn.metrics and then see if you can figure out how to use it to easily get a full report of your model's performance on the test data. **

In [23]:
y_pred = class_ids
target_names = ['class 0', 'class 1']

In [24]:
print(metrics.classification_report(y_test, y_pred, target_names=target_names))

             precision    recall  f1-score   support

    class 0       0.84      0.95      0.89      7436
    class 1       0.73      0.43      0.54      2333

avg / total       0.82      0.83      0.81      9769



# Great Job!